# Modelo

## Librerías

In [1]:
from pulp import *
import numpy as np
import pandas as pd


In [ ]:

# ======================
# DATA PREPARATION
# ======================

# Define sets
num_plants = 10
num_periods = 30  # e.g., 30 days
num_suppliers = 5
num_polygons = 31

plants = [f'Plant_{i}' for i in range(1, num_plants+1)]
periods = [f'Day_{j}' for j in range(1, num_periods+1)]
suppliers = [f'Supplier_{k}' for k in range(1, num_suppliers+1)]
polygons = [f'Polygon_{p}' for p in range(1, num_polygons+1)]

# Generate random data (replace with real data)
np.random.seed(42)

# Plant parameters
plant_volume = {p: np.random.uniform(0.01, 0.1) for p in plants}  # m³ per plant
plant_cost = {(p,s): np.random.uniform(5, 20) for p in plants for s in suppliers}  # cost per plant
plant_rest_time = {p: np.random.choice([20, 60]) for p in plants}  # minutes
plant_storage_min = {p: 72 for p in plants}  # hours (3 days)
plant_storage_max = {p: 168 for p in plants}  # hours (7 days)

# Supplier parameters
supplier_delivery_time = {s: np.random.randint(1, 5) for s in suppliers}  # days

# Transportation parameters
transport_cost = 4500  # per trip
truck_capacity = 1.238377  # m³
max_plants_per_trip = 8000

# Storage parameters
warehouse_capacity = 400  # m²
treatment_capacity = 100  # m²

# Planting parameters
planting_cost = {p: 20 for p in plants}  # cost to plant each plant
daily_hectares = 5
plants_per_hectare = 2000  # assumption

# Gasoline parameters
gasoline_cost = 23.97  # per liter
tank_capacity = 83  # liters
fuel_efficiency = 4  # km per liter

# Distances (random for example)
distances = {
    'warehouse': {p: np.random.uniform(5, 50) for p in polygons},
    'suppliers': {s: np.random.uniform(10, 100) for s in suppliers}
}

# Demand (random for example)
demand = {(p, poly): np.random.randint(100, 1000) 
          for p in plants for poly in polygons}

# ======================
# OPTIMIZATION MODEL
# ======================

# Create the model
model = LpProblem("Reforestation_Supply_Chain", LpMinimize)

# ======================
# DECISION VARIABLES
# ======================

# Transportation from suppliers to warehouse
Ct = LpVariable.dicts("Transport", 
                     ((i, j, k) for i in plants 
                                 for j in periods 
                                 for k in suppliers),
                     lowBound=0, cat='Integer')

# Number of trips
n = LpVariable("Number_of_Trips", lowBound=0, cat='Integer')

# Plants moved to treatment
CA = LpVariable.dicts("To_Treatment", 
                     ((i, j) for i in plants for j in periods),
                     lowBound=0, cat='Integer')

# Plants planted
CP = LpVariable.dicts("Planting", 
                     ((i, j, p) for i in plants 
                                for j in periods 
                                for p in polygons),
                     lowBound=0, cat='Integer')

# Treatment decision (binary)
x = LpVariable.dicts("Treatment", plants, cat='Binary')

# Gasoline used
L = LpVariable("Gasoline_Used", lowBound=0, cat='Continuous')

# Inventory variables
A = LpVariable.dicts("Inventory", 
                    ((i, j) for i in plants for j in ['Day_0'] + periods),
                    lowBound=0, cat='Integer')

# ======================
# OBJECTIVE FUNCTION
# ======================

# Cost minimization
cost_objective = (
    n * transport_cost + 
    lpSum(plant_cost[i,k] * Ct[i,j,k] for i in plants for j in periods for k in suppliers) +
    lpSum(planting_cost[i] * CP[i,j,p] for i in plants for j in periods for p in polygons) +
    L * gasoline_cost
)

# Survival maximization (we'll handle this as a constraint with target)
survival_weight = 0.3  # Weight for survival in multi-objective
# (This would need a proper survival probability model)

model += cost_objective  # Primary objective

# ======================
# CONSTRAINTS
# ======================

# Initial inventory
for i in plants:
    model += A[i, 'Day_0'] == 0

# Inventory balance
for i in plants:
    for j in periods:
        model += (
            A[i, j] == A[i, periods[periods.index(j)-1]] + 
            lpSum(Ct[i,j,k] for k in suppliers) - 
            CA[i,j]
        )

# Warehouse capacity
for j in periods:
    model += (
        lpSum(plant_volume[i] * A[i,j] for i in plants) <= warehouse_capacity
    )

# Treatment capacity
for j in periods:
    model += (
        lpSum(plant_volume[i] * CA[i,j] for i in plants) <= treatment_capacity
    )

# Minimum and maximum storage time
# (Simplified as constraints on when plants can leave storage)
for i in plants:
    for j in periods:
        if periods.index(j) >= 3:  # At least 3 days storage
            model += CA[i,j] <= A[i, periods[periods.index(j)-3]]
        if periods.index(j) >= 7:  # At most 7 days storage
            model += CA[i,j] >= A[i, periods[periods.index(j)-7]] - lpSum(CA[i,periods[k]] 
                     for k in range(periods.index(j)-7, periods.index(j)))

# Trip capacity constraints
for j in periods:
    model += (
        lpSum(Ct[i,j,k] for i in plants for k in suppliers) <= max_plants_per_trip * n
    )

# Planting demand satisfaction
for i in plants:
    for p in polygons:
        model += (
            lpSum(CP[i,j,p] for j in periods) == demand[i,p]
        )

# Daily planting requirement
for j in periods:
    model += (
        lpSum(CP[i,j,p] for i in plants for p in polygons) >= daily_hectares * plants_per_hectare
    )

# Truck capacity for planting trips
for j in periods:
    model += (
        lpSum(plant_volume[i] * CP[i,j,p] for i in plants for p in polygons) <= truck_capacity
    )

# Gasoline constraints
model += L <= tank_capacity * n
# (Simplified distance constraint)
model += (
    lpSum(CP[i,j,p] * distances['warehouse'][p] for i in plants for j in periods for p in polygons) / 
    fuel_efficiency <= L
)

# Treatment time constraints
for i in plants:
    if plant_rest_time[i] == 60:  # Needs treatment
        model += x[i] == 1
    else:
        model += x[i] <= 1  # Optional

# ======================
# SOLVE MODEL
# ======================

# Solve the model
model.solve(PULP_CBC_CMD(msg=True))

# ======================
# RESULTS ANALYSIS
# ======================

print(f"Status: {LpStatus[model.status]}")
print(f"Total Cost: ${value(model.objective):,.2f}")
print(f"Number of Trips: {value(n)}")
print(f"Gasoline Used: {value(L):.2f} liters")

# Transportation summary
transport_df = pd.DataFrame([
    {'Plant': i, 'Period': j, 'Supplier': k, 'Quantity': value(Ct[i,j,k])}
    for i in plants for j in periods for k in suppliers if value(Ct[i,j,k]) > 0
])

# Planting summary
planting_df = pd.DataFrame([
    {'Plant': i, 'Period': j, 'Polygon': p, 'Quantity': value(CP[i,j,p])}
    for i in plants for j in periods for p in polygons if value(CP[i,j,p]) > 0
])

print("\nTransportation Plan (first 10 rows):")
print(transport_df.head(10))

print("\nPlanting Plan (first 10 rows):")
print(planting_df.head(10))

# Calculate survival probability (example calculation)
# This would need a proper probabilistic model based on your data
if LpStatus[model.status] == 'Optimal':
    total_plants = sum(value(CP[i,j,p]) for i in plants for j in periods for p in polygons)
    avg_storage_time = sum(
        (periods.index(j)+1) * value(CA[i,j]) for i in plants for j in periods
    ) / total_plants if total_plants > 0 else 0
    
    # Simplified survival estimation
    survival_rate = 0.7 * (avg_storage_time - 72) / (168 - 72) if avg_storage_time > 72 else 0.5
    print(f"\nEstimated Average Survival Rate: {survival_rate:.2%}")

# Data

In [8]:
planting_sequence = {
    1: 'E2', 2: 'E9', 3: 'E8', 4: 'E2', 5: 'E7', 
    6: 'E3', 7: 'E5', 8: 'E2', 9: 'E6', 10: 'E4',
    11: 'E2', 12: 'E6', 13: 'E9', 14: 'E2', 15: 'E5',
    16: 'E1', 17: 'E2', 18: 'E8', 19: 'E10', 20: 'E2',
    21: 'E7', 22: 'E9'
}

In [11]:
survival_matrix = {
    'E1': [0.6, 0.7, 0.8, 0.9, 0.95],
    'E2': [0.65, 0.75, 0.85, 0.92, 0.97],
    'E3': [0.6, 0.7, 0.8, 0.88, 0.95],
    'E4': [0.58, 0.68, 0.78, 0.87, 0.94],
    'E5': [0.5, 0.65, 0.8, 0.9, 0.98],
    'E6': [0.52, 0.66, 0.82, 0.91, 0.98],
    'E7': [0.55, 0.68, 0.83, 0.92, 0.99],
    'E8': [0.54, 0.67, 0.82, 0.91, 0.98],
    'E9': [0.4, 0.5, 0.6, 0.75, 0.85],
    'E10': [0.55, 0.65, 0.75, 0.85, 0.93]
}

In [12]:
#Viveros
nursery_availability = {
    'E1': {'V1': 0, 'V2': 0, 'V3': 0, 'V4': 26},
    'E2': {'V1': 0, 'V2': 0, 'V3': 0, 'V4': 26},
    'E3': {'V1': 0, 'V2': 26, 'V3': 0, 'V4': 26},
    'E4': {'V1': 0, 'V2': 26, 'V3': 25, 'V4': 0},
    'E5': {'V1': 0, 'V2': 17, 'V3': 18, 'V4': 0},
    'E6': {'V1': 0, 'V2': 0, 'V3': 18, 'V4': 21},
    'E7': {'V1': 0, 'V2': 17, 'V3': 18, 'V4': 18},
    'E8': {'V1': 0, 'V2': 0, 'V3': 18, 'V4': 0},
    'E9': {'V1': 26.5, 'V2': 0, 'V3': 0, 'V4': 0},
    'E10': {'V1': 26, 'V2': 0, 'V3': 0, 'V4': 0}
}


In [13]:
plant_volumes = {
    'E1': 0.025,
    'E2': 0.025,
    'E3': 0.0125,
    'E4': 0.0171875,
    'E5': 0.015625,
    'E6': 0.015625,
    'E7': 0.015625,
    'E8': 0.0171875,
    'E9': 0.0171875,
    'E10': 0.015625
}

In [14]:
polygon_demands = {
    '27': {'E1': 4, 'E2': 17, 'E3': 4, 'E4': 4, 'E5': 4, 'E6': 3, 'E7': 6, 'E8': 6, 'E9': 8, 'E10': 2},
    '28': {'E1': 31, 'E2': 148, 'E3': 31, 'E4': 31, 'E5': 37, 'E6': 28, 'E7': 55, 'E8': 48, 'E9': 65, 'E10': 20},
    '29': {'E1': 30, 'E2': 142, 'E3': 30, 'E4': 30, 'E5': 35, 'E6': 27, 'E7': 53, 'E8': 46, 'E9': 63, 'E10': 19},
    '30': {'E1': 34, 'E2': 162, 'E3': 34, 'E4': 34, 'E5': 40, 'E6': 31, 'E7': 60, 'E8': 53, 'E9': 71, 'E10': 22},
    '20': {'E1': 4, 'E2': 21, 'E3': 4, 'E4': 4, 'E5': 5, 'E6': 4, 'E7': 8, 'E8': 7, 'E9': 9, 'E10': 3},
    '21': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24},
    '22': {'E1': 36, 'E2': 170, 'E3': 36, 'E4': 36, 'E5': 42, 'E6': 32, 'E7': 63, 'E8': 55, 'E9': 75, 'E10': 23},
    '23': {'E1': 26, 'E2': 121, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 39, 'E9': 53, 'E10': 16},
    '24': {'E1': 26, 'E2': 122, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 40, 'E9': 54, 'E10': 16},
    '18': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21},
    '17': {'E1': 27, 'E2': 128, 'E3': 27, 'E4': 27, 'E5': 32, 'E6': 24, 'E7': 47, 'E8': 41, 'E9': 56, 'E10': 17},
    '16': {'E1': 25, 'E2': 118, 'E3': 25, 'E4': 25, 'E5': 29, 'E6': 23, 'E7': 44, 'E8': 38, 'E9': 52, 'E10': 16},
    '19': {'E1': 21, 'E2': 102, 'E3': 21, 'E4': 21, 'E5': 25, 'E6': 20, 'E7': 38, 'E8': 33, 'E9': 45, 'E10': 14},
    '25': {'E1': 23, 'E2': 110, 'E3': 23, 'E4': 23, 'E5': 27, 'E6': 21, 'E7': 41, 'E8': 36, 'E9': 48, 'E10': 15},
    '26': {'E1': 22, 'E2': 106, 'E3': 22, 'E4': 22, 'E5': 26, 'E6': 20, 'E7': 39, 'E8': 34, 'E9': 46, 'E10': 14},
    '13': {'E1': 36, 'E2': 172, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 63, 'E8': 56, 'E9': 75, 'E10': 23},
    '31': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 37, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20},
    '14': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18},
    '15': {'E1': 25, 'E2': 120, 'E3': 25, 'E4': 25, 'E5': 30, 'E6': 23, 'E7': 44, 'E8': 39, 'E9': 53, 'E10': 16},
    '7': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18},
    '8': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21},
    '9': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24},
    '10': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23},
    '11': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23},
    '12': {'E1': 6, 'E2': 31, 'E3': 6, 'E4': 6, 'E5': 8, 'E6': 6, 'E7': 11, 'E8': 10, 'E9': 13, 'E10': 4},
    '6': {'E1': 18, 'E2': 86, 'E3': 18, 'E4': 18, 'E5': 21, 'E6': 16, 'E7': 32, 'E8': 28, 'E9': 38, 'E10': 12},
    '2': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 38, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20},
    '3': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23},
    '4': {'E1': 35, 'E2': 168, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 55, 'E9': 74, 'E10': 22},
    '5': {'E1': 35, 'E2': 167, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 54, 'E9': 74, 'E10': 22},
    '1': {'E1': 24, 'E2': 115, 'E3': 24, 'E4': 24, 'E5': 29, 'E6': 22, 'E7': 42, 'E8': 37, 'E9': 50, 'E10': 15}
}


In [17]:
# Convert to more usable format
plants = list(survival_matrix.keys())
suppliers = ['V1', 'V2', 'V3', 'V4']
polygons = list(polygon_demands.keys())
periods = [f'Day_{d}' for d in range(1, 100)]  # 30-day planning horizon

# Additional parameters
transport_cost = 4500  # per trip
planting_cost = 20  # per plant
warehouse_capacity = 400  # m³
treatment_capacity = 100  # m³
truck_capacity = 1.238377  # m³
max_plants_per_trip = 8000
daily_hectares = 5
plants_per_hectare = 2000  # assumption
gasoline_cost = 23.97  # per liter
tank_capacity = 83  # liters
fuel_efficiency = 4  # km per liter

In [19]:
model = LpProblem("Reforestation_Optimization", LpMinimize)


In [24]:



# ======================
# OPTIMIZATION MODEL
# ======================


# ======================
# DECISION VARIABLES
# ======================

# Transportation from nurseries to warehouse
Ct = LpVariable.dicts("Transport", 
                     ((i, j, k) for i in plants 
                                 for j in periods 
                                 for k in suppliers),
                     lowBound=0, cat='Integer')

# Number of trips
n = LpVariable("Number_of_Trips", lowBound=0, cat='Integer')

# Plants moved to treatment
CA = LpVariable.dicts("To_Treatment", 
                     ((i, j) for i in plants for j in periods),
                     lowBound=0, cat='Integer')

# Plants planted
CP = LpVariable.dicts("Planting", 
                     ((i, j, p) for i in plants 
                                for j in periods 
                                for p in polygons),
                     lowBound=0, cat='Integer')

# Inventory variables
A = LpVariable.dicts("Inventory", 
                    ((i, j) for i in plants for j in ['Day_0'] + periods),
                    lowBound=0, cat='Integer')

# Storage days (for survival calculation)
storage_days = LpVariable.dicts("Storage_Days",
                              ((i, j) for i in plants for j in periods),
                              lowBound=3, upBound=7, cat='Continuous')

# ======================
# OBJECTIVE FUNCTION
# ======================

from pulp import lpSum, value  # Ensure lpSum and value are imported

# Cost components
transport_cost_total = n * transport_cost
purchase_cost_total = lpSum(Ct[i,j,k] * 10 for i in plants for j in periods for k in suppliers)  # Assuming 10 pesos per plant
planting_cost_total = lpSum(CP[i,j,p] * planting_cost for i in plants for j in periods for p in polygons)

# Survival component (to maximize)
survival_total = lpSum(
    CP[i,j,p] * survival_matrix[i][min(max(int(value(storage_days[i,j]) or 0)-3, 0), 4)]  # Cap at day 7
    if value(storage_days[i,j]) is not None else 0
    for i in plants for j in periods for p in polygons
)

# Combined objective (minimize cost, maximize survival)
model += transport_cost_total + purchase_cost_total + planting_cost_total - 0.1 * survival_total

# ======================
# CONSTRAINTS
# ======================

# Initial inventory
for i in plants:
    model += A[i, 'Day_0'] == 0

# Inventory balance
for i in plants:
    for j in periods:
        model += A[i, j] == A[i, periods[periods.index(j)-1]] + lpSum(Ct[i,j,k] for k in suppliers) - CA[i,j]

# Warehouse capacity
for j in periods:
    model += lpSum(plant_volumes[i] * A[i,j] for i in plants) <= warehouse_capacity

# Treatment capacity
for j in periods:
    model += lpSum(plant_volumes[i] * CA[i,j] for i in plants) <= treatment_capacity

# Nursery availability
for i in plants:
    for k in suppliers:
        model += lpSum(Ct[i,j,k] for j in periods) <= nursery_availability[i][k] * 1000  # Convert to plants

# Planting sequence constraint (simplified - ensure proper ratios)
sequence_counts = {i: list(planting_sequence.values()).count(i) for i in plants}
total_positions = len(planting_sequence)
for i in plants:
    ratio = sequence_counts[i] / total_positions
    for p in polygons:
        total_demand = lpSum(CP[i,j,p] for j in periods)
        model += total_demand >= ratio * polygon_demands[p][i] * 0.9  # Allow 10% flexibility
        model += total_demand <= ratio * polygon_demands[p][i] * 1.1

# Demand satisfaction
for i in plants:
    for p in polygons:
        model += lpSum(CP[i,j,p] for j in periods) == polygon_demands[p][i]

# Storage days calculation (simplified)
for i in plants:
    for j in periods[3:]:  # Starting from day 4
        model += storage_days[i,j] >= 3
        model += storage_days[i,j] <= 7
        # Linear approximation of survival probability
        model += survival_matrix[i][0] + (storage_days[i,j]-3)*(survival_matrix[i][4]-survival_matrix[i][0])/4 >= 0

# Truck capacity for planting
for j in periods:
    model += lpSum(plant_volumes[i] * CP[i,j,p] for i in plants for p in polygons) <= truck_capacity

# Daily planting requirement
for j in periods:
    model += lpSum(CP[i,j,p] for i in plants for p in polygons) >= daily_hectares * plants_per_hectare

# ======================
# SOLVE MODEL
# ======================

# Ensure the CBC solver is installed and explicitly specify the solver path
from pulp import PULP_CBC_CMD

solver = PULP_CBC_CMD(path=pulp_cbc_path, msg=True)
model.solve(solver)

# ======================
# RESULTS ANALYSIS
# ======================

print(f"Status: {LpStatus[model.status]}")
print(f"Total Cost: ${value(model.objective):,.2f}")
print(f"Transport Cost: ${value(transport_cost_total):,.2f}")
print(f"Planting Cost: ${value(planting_cost_total):,.2f}")
print(f"Number of Trips: {value(n)}")

# Calculate actual survival rate
total_plants = sum(value(CP[i,j,p]) for i in plants for j in periods for p in polygons)
avg_survival = sum(
    value(CP[i, j, p]) * survival_matrix[i][min(max(int(value(storage_days[i, j]) or 0) - 3, 0), 4)]
    if value(storage_days[i, j]) is not None else 0
    for i in plants for j in periods for p in polygons
) / total_plants
print(f"Average Survival Rate: {avg_survival:.2%}")

# Output planting schedule
planting_schedule = pd.DataFrame([
    {
        'Plant': i,
        'Period': j,
        'Polygon': p,
        'Quantity': value(CP[i, j, p]),
        'StorageDays': value(storage_days[i, j]) if value(storage_days[i, j]) is not None else 0,
        'SurvivalProb': survival_matrix[i][min(max(int(value(storage_days[i, j]) or 0) - 3, 0), 4)]
        if value(storage_days[i, j]) is not None else 0
    }
    for i in plants for j in periods for p in polygons if value(CP[i, j, p]) > 0
])

print("\nPlanting Schedule (first 10 rows):")
print(planting_schedule.head(10))

print("\nPlanting Schedule (first 10 rows):")
print(planting_schedule.head(10))

PulpSolverError: Use COIN_CMD if you want to set a path

In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd

# Load your image
image = cv2.imread("IMG_14.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib

# Plant color mapping (create distinct colors for each species)
plant_colors = {
    'E1': (255, 0, 0),     # Red
    'E2': (0, 255, 0),     # Green
    'E3': (0, 0, 255),     # Blue
    'E4': (255, 255, 0),   # Yellow
    'E5': (255, 0, 255),   # Magenta
    'E6': (0, 255, 255),   # Cyan
    'E7': (128, 0, 0),     # Maroon
    'E8': (0, 128, 0),     # Dark Green
    'E9': (0, 0, 128),     # Navy
    'E10': (128, 128, 0)   # Olive
}

# Create visualization function
def visualize_planting(image, contours, planting_schedule, scale_factor, espaciado_plantas):
    # Create a copy of the image to draw on
    viz_image = image.copy()
    height, width = viz_image.shape[:2]
    
    # Process each polygon
    for poly_id, cnt in enumerate(contours):
        poly_num = str(poly_id + 1)  # Match your polygon numbering
        
        if poly_num not in planting_schedule['Polygon'].unique():
            continue
            
        # Create mask for this polygon
        mask = np.zeros((height, width), dtype=np.uint8)
        cv2.drawContours(mask, [cnt], 0, 255, -1)
        
        # Get spacing in pixels
        spacing_px = int(espaciado_plantas * scale_factor)
        
        # Generate planting positions (3 bolillos pattern)
        plant_positions = []
        for y in range(0, height, spacing_px):
            offset = (spacing_px // 2) if (y // spacing_px) % 2 == 1 else 0
            for x in range(0 + offset, width, spacing_px):
                if mask[y, x] == 255:
                    plant_positions.append((x, y))
        
        # Get planting schedule for this polygon
        poly_schedule = planting_schedule[planting_schedule['Polygon'] == poly_num]
        
        # Assign plants to positions according to sequence
        seq_idx = 0
        day_colors = {}  # To store day groupings
        
        for _, row in poly_schedule.iterrows():
            plant_type = row['Plant']
            quantity = int(row['Quantity'])
            day = row['Day']
            
            # Create color that also encodes day (lighter = later day)
            day_num = int(day.split('_')[1])
            day_factor = min(0.5 + (day_num / 60), 0.8)  # Lighten color based on day
            
            base_color = np.array(plant_colors[plant_type]) / 255.0
            day_color = base_color * day_factor
            day_color = (day_color * 255).astype(np.uint8)
            
            # Plant the required quantity
            for _ in range(quantity):
                if seq_idx >= len(plant_positions):
                    break
                
                x, y = plant_positions[seq_idx]
                
                # Draw the plant (larger circle for visualization)
                cv2.circle(viz_image, (x, y), 5, day_color.tolist(), -1)
                
                # Small black border for contrast
                cv2.circle(viz_image, (x, y), 6, (0, 0, 0), 1)
                
                seq_idx += 1
        
    return viz_image

# Assuming you have these from your optimization results
# planting_schedule should be a DataFrame with columns: Plant, Day, Polygon, Quantity

# Example planting schedule (replace with your actual results)
planting_schedule = pd.DataFrame([
    {'Plant': 'E2', 'Day': 'Day_1', 'Polygon': '27', 'Quantity': 5},
    {'Plant': 'E9', 'Day': 'Day_1', 'Polygon': '27', 'Quantity': 3},
    # ... add all your planting schedule results here
])

# Generate visualization
result_image = visualize_planting(image, contours, planting_schedule, scale_factor, espaciado_plantas)

# Create legend
plt.figure(figsize=(15, 10))
plt.imshow(result_image)
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=f'{plant}', 
                   markerfacecolor=np.array(color)/255.0, markersize=10)
                  for plant, color in plant_colors.items()]
plt.legend(handles=legend_elements, title='Plant Species', bbox_to_anchor=(1.05, 1), loc='upper left')

# Add day progression explanation
plt.text(width + 100, height//2, "Color Intensity Indicates Planting Day\n(Darker = Earlier, Lighter = Later)", 
         ha='left', va='center')

plt.axis('off')
plt.tight_layout()
plt.savefig('planting_visualization.jpg', bbox_inches='tight', dpi=300)
plt.show()

[ WARN:0@0.393] global loadsave.cpp:268 findDecoder imread_('IMG_14.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


# Model

In [7]:
import pulp
import pandas as pd
import numpy as np
from itertools import product

# ======================
# DATOS DE ENTRADA
# ======================

# Especies y sus características
especies = {
    'E1': {'nombre': 'Agave lechuguilla', 'volumen': 0.025, 'nopal': 0, 'tratamiento_min': 20},
    'E2': {'nombre': 'Agave salmiana', 'volumen': 0.025, 'nopal': 0, 'tratamiento_min': 20},
    'E3': {'nombre': 'Agave scabra', 'volumen': 0.0125, 'nopal': 0, 'tratamiento_min': 20},
    'E4': {'nombre': 'Agave striata', 'volumen': 0.0171875, 'nopal': 0, 'tratamiento_min': 20},
    'E5': {'nombre': 'Opuntia cantabrigiensis', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E6': {'nombre': 'Opuntia engelmani', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E7': {'nombre': 'Opuntia robusta', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E8': {'nombre': 'Opuntia streptacanta', 'volumen': 0.0171875, 'nopal': 1, 'tratamiento_min': 60},
    'E9': {'nombre': 'Prosopis laevigata', 'volumen': 0.0171875, 'nopal': 0, 'tratamiento_min': 20},
    'E10': {'nombre': 'Yucca filifera', 'volumen': 0.015625, 'nopal': 0, 'tratamiento_min': 20}
}

# Viveros y disponibilidad (en miles de plantas)
viveros = {
    'V1': {'E9': 26.5, 'E10': 26},
    'V2': {'E3': 26, 'E4': 26, 'E5': 17, 'E7': 17},
    'V3': {'E4': 25, 'E5': 18, 'E6': 18, 'E7': 18, 'E8': 18},
    'V4': {'E1': 26, 'E2': 26, 'E3': 26, 'E6': 21, 'E7': 18 }
}

# Polígonos y demanda
poligonos = {
    '27': {'hectareas': 1.28, 'demanda': {'E1': 4, 'E2': 17, 'E3': 4, 'E4': 4, 'E5': 4, 'E6': 3, 'E7': 6, 'E8': 6, 'E9': 8, 'E10': 2}},
    '28': {'hectareas': 6.64, 'demanda': {'E1': 31, 'E2': 148, 'E3': 31, 'E4': 31, 'E5': 37, 'E6': 28, 'E7': 55, 'E8': 48, 'E9': 65, 'E10': 20}},
    '29': {'hectareas': 6.54, 'demanda': {'E1': 30, 'E2': 142, 'E3': 30, 'E4': 30, 'E5': 35, 'E6': 27, 'E7': 53, 'E8': 46, 'E9': 63, 'E10': 19}},
    '30': {'hectareas': 6.76, 'demanda': {'E1': 34, 'E2': 162, 'E3': 34, 'E4': 34, 'E5': 40, 'E6': 31, 'E7': 60, 'E8': 53, 'E9': 71, 'E10': 22}},
    '20': {'hectareas': 1.38, 'demanda': {'E1': 4, 'E2': 21, 'E3': 4, 'E4': 4, 'E5': 5, 'E6': 4, 'E7': 8, 'E8': 7, 'E9': 9, 'E10': 3}},
    '21': {'hectareas': 8.0, 'demanda': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24}},
    '22': {'hectareas': 7.82, 'demanda': {'E1': 36, 'E2': 170, 'E3': 36, 'E4': 36, 'E5': 42, 'E6': 32, 'E7': 63, 'E8': 55, 'E9': 75, 'E10': 23}},
    '23': {'hectareas': 5.53, 'demanda': {'E1': 26, 'E2': 121, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 39, 'E9': 53, 'E10': 16}},
    '24': {'hectareas': 5.64, 'demanda': {'E1': 26, 'E2': 122, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 40, 'E9': 54, 'E10': 16}},
    '18': {'hectareas': 7.11, 'demanda': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21}},
    '17': {'hectareas': 6.11, 'demanda': {'E1': 27, 'E2': 128, 'E3': 27, 'E4': 27, 'E5': 32, 'E6': 24, 'E7': 47, 'E8': 41, 'E9': 56, 'E10': 17}},
    '16': {'hectareas': 5.64, 'demanda': {'E1': 25, 'E2': 118, 'E3': 25, 'E4': 25, 'E5': 29, 'E6': 23, 'E7': 44, 'E8': 38, 'E9': 52, 'E10': 16}},
    '19': {'hectareas': 4.92, 'demanda': {'E1': 21, 'E2': 102, 'E3': 21, 'E4': 21, 'E5': 25, 'E6': 20, 'E7': 38, 'E8': 33, 'E9': 45, 'E10': 14}},
    '25': {'hectareas': 5.05, 'demanda': {'E1': 23, 'E2': 110, 'E3': 23, 'E4': 23, 'E5': 27, 'E6': 21, 'E7': 41, 'E8': 36, 'E9': 48, 'E10': 15}},
    '26': {'hectareas': 4.75, 'demanda': {'E1': 22, 'E2': 106, 'E3': 22, 'E4': 22, 'E5': 26, 'E6': 20, 'E7': 39, 'E8': 34, 'E9': 46, 'E10': 14}},
    '13': {'hectareas': 7.97, 'demanda': {'E1': 36, 'E2': 172, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 63, 'E8': 56, 'E9': 75, 'E10': 23}},
    '31': {'hectareas': 7.34, 'demanda': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 37, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20}},
    '14': {'hectareas': 5.98, 'demanda': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18}},
    '15': {'hectareas': 5.4, 'demanda': {'E1': 25, 'E2': 120, 'E3': 25, 'E4': 25, 'E5': 30, 'E6': 23, 'E7': 44, 'E8': 39, 'E9': 53, 'E10': 16}},
    '7': {'hectareas': 6.28, 'demanda': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18}},
    '8': {'hectareas': 7.6, 'demanda': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21}},
    '9': {'hectareas': 8.0, 'demanda': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24}},
    '10': {'hectareas': 8.0, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '11': {'hectareas': 7.67, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '12': {'hectareas': 1.47, 'demanda': {'E1': 6, 'E2': 31, 'E3': 6, 'E4': 6, 'E5': 8, 'E6': 6, 'E7': 11, 'E8': 10, 'E9': 13, 'E10': 4}},
    '6': {'hectareas': 4.19, 'demanda': {'E1': 18, 'E2': 86, 'E3': 18, 'E4': 18, 'E5': 21, 'E6': 16, 'E7': 32, 'E8': 28, 'E9': 38, 'E10': 12}},
    '2': {'hectareas': 7.52, 'demanda': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 38, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20}},
    '3': {'hectareas': 8.0, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '4': {'hectareas': 8.0, 'demanda': {'E1': 35, 'E2': 168, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 55, 'E9': 74, 'E10': 22}},
    '5': {'hectareas': 7.56, 'demanda': {'E1': 35, 'E2': 167, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 54, 'E9': 74, 'E10': 22}},
    '1': {'hectareas': 5.4, 'demanda': {'E1': 24, 'E2': 115, 'E3': 24, 'E4': 24, 'E5': 29, 'E6': 22, 'E7': 42, 'E8': 37, 'E9': 50, 'E10': 15}},
}




# Parámetros logísticos
parametros = {
    'costo_transporte': 4500,  # $ por viaje
    'costo_plantacion': 20,    # $ por planta
    'capacidad_almacen': 400,  # m³
    'capacidad_tratamiento': 100,  # m³
    'capacidad_camion': 1.238377,  # m³
    'max_plantas_viaje': 8000,
    'hectareas_dia': 5,
    'plantas_hectarea': 2000,
    'jornada_laboral': 6,  # horas
    'costo_gasolina': 23.97,  # $/litro
    'rendimiento_gasolina': 4  # km/litro
}

# Secuencia de plantación (posiciones 1-22)
secuencia_plantacion = [
    'E2', 'E9', 'E8', 'E2', 'E7', 'E3', 'E5', 'E2', 'E6', 'E4',
    'E2', 'E6', 'E9', 'E2', 'E5', 'E1', 'E2', 'E8', 'E10', 'E2',
    'E7', 'E9'
]

# Matriz de supervivencia (días 3-7)
supervivencia = {
    'E1': [0.6, 0.7, 0.8, 0.9, 0.95],
    'E2': [0.65, 0.75, 0.85, 0.92, 0.97],
    'E3': [0.6, 0.7, 0.8, 0.88, 0.95],
    'E4': [0.58, 0.68, 0.78, 0.87, 0.94],
    'E5': [0.5, 0.65, 0.8, 0.9, 0.98],
    'E6': [0.52, 0.66, 0.82, 0.91, 0.98],
    'E7': [0.55, 0.68, 0.83, 0.92, 0.99],
    'E8': [0.54, 0.67, 0.82, 0.91, 0.98],
    'E9': [0.4, 0.5, 0.6, 0.75, 0.85],
    'E10': [0.55, 0.65, 0.75, 0.85, 0.93]
}

# ======================
# MODELO DE OPTIMIZACIÓN
# ======================

modelo = pulp.LpProblem("Optimizacion_Cadena_Suministro_Reforestacion", pulp.LpMinimize)

# ======================
# VARIABLES DE DECISIÓN
# ======================

# 1. Transporte desde viveros
X = pulp.LpVariable.dicts(
    "Transporte",
    ((e, d, v) for e in especies for d in range(1, 31) for v in viveros),
    lowBound=0,
    cat='Integer'
)

# 2. Plantas en almacén
A = pulp.LpVariable.dicts(
    "Almacen",
    ((e, d) for e in especies for d in range(0, 31)),
    lowBound=0,
    cat='Integer'
)

# 3. Plantas enviadas a tratamiento
T = pulp.LpVariable.dicts(
    "Tratamiento",
    ((e, d) for e in especies for d in range(1, 31)),
    lowBound=0,
    cat='Integer'
)

# 4. Plantas plantadas
P = pulp.LpVariable.dicts(
    "Plantacion",
    ((e, d, p) for e in especies for d in range(1, 31) for p in poligonos),
    lowBound=0,
    cat='Integer'
)

# 5. Viajes de transporte
N = pulp.LpVariable("Viajes", lowBound=0, cat='Integer')

# 6. Gasolina utilizada
G = pulp.LpVariable("Gasolina", lowBound=0, cat='Continuous')

# 7. Días de almacenamiento
D = pulp.LpVariable.dicts(
    "Dias_Almacen",
    ((e, d) for e in especies for d in range(1, 31)),
    lowBound=3,
    upBound=7,
    cat='Continuous'
)

# ======================
# FUNCIÓN OBJETIVO
# ======================

# Costos
costo_total = (
    N * parametros['costo_transporte'] +
    pulp.lpSum(X[e, d, v] * 10 for e in especies for d in range(1, 31) for v in viveros) +  # Costo plantas
    pulp.lpSum(P[e, d, p] * parametros['costo_plantacion'] for e in especies for d in range(1, 31) for p in poligonos) +
    G * parametros['costo_gasolina']
)

# Supervivencia (maximizar)
supervivencia_total = pulp.lpSum(
    P[e, d, p] * supervivencia[e][min(int(pulp.value(D[e, d]) or 3), 7)-3]  # Índice 0-4 para días 3-7
    for e in especies for d in range(1, 31) for p in poligonos
)

modelo += costo_total - 0.1 * supervivencia_total  # Ponderación

# ======================
# RESTRICCIONES
# ======================

# 1. Inventario inicial
for e in especies:
    modelo += A[e, 0] == 0

# 2. Balance de inventario
for e in especies:
    for d in range(1, 31):
        modelo += A[e, d] == A[e, d-1] + pulp.lpSum(X[e, d, v] for v in viveros) - T[e, d]

# 3. Capacidad de almacén
for d in range(1, 31):
    modelo += pulp.lpSum(A[e, d] * especies[e]['volumen'] for e in especies) <= parametros['capacidad_almacen']

# 4. Disponibilidad en viveros
for v in viveros:
    for e in especies:
        if e in viveros[v]:
            modelo += pulp.lpSum(X[e, d, v] for d in range(1, 31)) <= viveros[v][e] * 1000

# 5. Secuencia de plantación
for p in poligonos:
    total_plantas = sum(poligonos[p]['demanda'].values())
    for pos, e in enumerate(secuencia_plantacion):
        ratio = secuencia_plantacion.count(e) / len(secuencia_plantacion)
        modelo += pulp.lpSum(P[e, d, p] for d in range(1, 31)) >= ratio * poligonos[p]['demanda'][e] * 0.9
        modelo += pulp.lpSum(P[e, d, p] for d in range(1, 31)) <= ratio * poligonos[p]['demanda'][e] * 1.1

# 6. Satisfacción de demanda
for p in poligonos:
    for e in especies:
        modelo += pulp.lpSum(P[e, d, p] for d in range(1, 31)) == poligonos[p]['demanda'][e]

# 7. Capacidad diaria de plantación
for d in range(1, 31):
    modelo += pulp.lpSum(P[e, d, p] for e in especies for p in poligonos) >= parametros['hectareas_dia'] * parametros['plantas_hectarea']

# 8. Tiempos de tratamiento
for e in especies:
    for d in range(1, 31):
        if especies[e]['nopal'] == 1:
            modelo += D[e, d] >= .33  #  20 minutos ≈ 0.33 horas
        else:
            modelo += D[e, d] >= 1  # 60 minutos = 1 hora (simplificado)

# 9. Restricciones de transporte
modelo += pulp.lpSum(X[e, d, v] for e in especies for d in range(1, 31) for v in viveros) <= parametros['max_plantas_viaje'] * N

# ======================
# RESOLUCIÓN
# ======================

modelo.solve(pulp.PULP_CBC_CMD(msg=True))

# ======================
# ANÁLISIS DE RESULTADOS
# ======================

if modelo.status == pulp.LpStatusOptimal:
    print("\nSOLUCIÓN ÓPTIMA ENCONTRADA")
    print(f"Costo total: ${pulp.value(costo_total):,.2f}")
    print(f"Viajes requeridos: {pulp.value(N)}")
    print(f"Gasolina utilizada: {pulp.value(G):.2f} litros")
    
    # Exportar resultados a CSV
    resultados = []
    for e in especies:
        for d in range(1, 31):
            for p in poligonos:
                dias_almacen = pulp.value(D[e, d]) or 3  # Default to 3 if None
                prob_supervivencia = supervivencia[e][min(int(dias_almacen), 7)-3]
                if pulp.value(P[e, d, p]) > 0:
                    resultados.append({
                        'Dia': d,
                        'Especie': e,
                        'Poligono': p,
                        'Cantidad': pulp.value(P[e, d, p]),
                        'Dias_Almacen': dias_almacen,
                        'Prob_Supervivencia': prob_supervivencia
                    })
    
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('plan_plantacion_optimizado.csv', index=False)
    print("Plan de plantación exportado a 'plan_plantacion_optimizado.csv'")
else:
    print("No se encontró solución óptima")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/715aa590ddf0457993b9b75832ebd159-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/715aa590ddf0457993b9b75832ebd159-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2366 COLUMNS
At line 99002 RHS
At line 101364 BOUNDS
At line 113076 ENDATA
Problem MODEL has 2361 rows, 11412 columns and 63911 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.02 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.05   (Wallclock seconds):       0.07

No se encontró solución óptima


In [11]:
# ======================
# CONFIGURACIÓN PRINCIPAL (MODIFICADA)
# ======================
dias_planificacion = range(1, 366)  # Cambio a 365 días
semanas_planificacion = range(1, 53)
meses_planificacion = range(1, 13)

# ======================
# VARIABLES DE DECISIÓN (ACTUALIZADAS)
# ======================
X = pulp.LpVariable.dicts(
    "Transporte",
    ((e, d, v) for e in especies for d in dias_planificacion for v in viveros),
    lowBound=0,
    cat='Integer'
)

A = pulp.LpVariable.dicts(
    "Almacen",
    ((e, d) for e in especies for d in range(0, 366)),  # 0 a 365
    lowBound=0,
    cat='Integer'
)

T = pulp.LpVariable.dicts(
    "Tratamiento",
    ((e, d) for e in especies for d in dias_planificacion),
    lowBound=0,
    cat='Integer'
)

P = pulp.LpVariable.dicts(
    "Plantacion",
    ((e, d, p) for e in especies for d in dias_planificacion for p in poligonos),
    lowBound=0,
    cat='Integer'
)

# Variables adicionales para manejar el año completo
N_mensual = pulp.LpVariable.dicts(
    "Viajes_Mensuales",
    (m for m in meses_planificacion),
    lowBound=0,
    cat='Integer'
)

D = pulp.LpVariable.dicts(
    "Dias_Almacen",
    ((e, d) for e in especies for d in dias_planificacion),
    lowBound=3,
    upBound=7,
    cat='Continuous'
)

# ======================
# FUNCIÓN OBJETIVO (OPTIMIZADA)
# ======================
# Costos anualizados con descuento por escala
costo_total = (
    pulp.lpSum(N_mensual[m] * parametros['costo_transporte'] * (0.95 if m in [11,12,1,2] else 1)  # 5% descuento en invierno
              for m in meses_planificacion) +
    pulp.lpSum(X[e, d, v] * 10 * (0.9 if d > 180 else 1)  # 10% descuento en segunda mitad del año
              for e in especies for d in dias_planificacion for v in viveros) +
    pulp.lpSum(P[e, d, p] * parametros['costo_plantacion'] 
              for e in especies for d in dias_planificacion for p in poligonos) +
    G * parametros['costo_gasolina'] * 0.9  # 10% descuento por compra anual
)

# Supervivencia con ajuste estacional
def prob_supervivencia_ajustada(e, dia):
    mes = (dia - 1) // 30 + 1
    base = supervivencia[e][min(int(pulp.value(D[e, dia]) or 3) - 3, 4)]
    if mes in [6,7,8]:  # Verano
        return base * 0.85  # 15% menos en meses calurosos
    elif mes in [12,1,2]:  # Invierno
        return base * 1.05  # 5% más en invierno
    return base

supervivencia_total = pulp.lpSum(
    P[e, d, p] * prob_supervivencia_ajustada(e, d)
    for e in especies for d in dias_planificacion for p in poligonos
)

modelo += costo_total - 0.15 * supervivencia_total  # Mayor peso a supervivencia

# ======================
# RESTRICCIONES ACTUALIZADAS
# ======================

# 1. Balance de inventario por temporadas
for e in especies:
    for d in dias_planificacion:
        # Suministros solo en temporadas específicas para ciertas especies
        if e in ['E9','E10'] and (d < 60 or d > 300):  # Invierno
            modelo += pulp.lpSum(X[e,d,v] for v in viveros) == 0
        
        modelo += A[e, d] == A[e, d-1] + pulp.lpSum(X[e, d, v] for v in viveros) - T[e, d]

# 2. Capacidad de almacenamiento variable
capacidad_estacional = {
    'alta': 400,  # m³
    'baja': 600   # m³ en temporada baja
}

for d in dias_planificacion:
    mes = (d - 1) // 30 + 1
    if mes in [6,7,8]:  # Verano - mayor capacidad
        modelo += pulp.lpSum(A[e, d] * especies[e]['volumen'] for e in especies) <= capacidad_estacional['baja']
    else:
        modelo += pulp.lpSum(A[e, d] * especies[e]['volumen'] for e in especies) <= capacidad_estacional['alta']

# 3. Secuencia de plantación con flexibilidad estacional
for p in poligonos:
    for e in especies:
        # Permitir ±15% de variación en la secuencia según temporada
        modelo += pulp.lpSum(P[e, d, p] for d in dias_planificacion) >= poligonos[p]['demanda'][e] * 0.85
        modelo += pulp.lpSum(P[e, d, p] for d in dias_planificacion) <= poligonos[p]['demanda'][e] * 1.15

# 4. Restricciones de transporte mensuales
for m in meses_planificacion:
    dias_mes = range((m-1)*30+1, min(m*30+1, 366))
    modelo += pulp.lpSum(X[e, d, v] for e in especies for d in dias_mes for v in viveros) <= parametros['max_plantas_viaje'] * N_mensual[m]

# 5. Requerimientos de plantación semanales
for semana in semanas_planificacion:
    dias_semana = range((semana-1)*7+1, min(semana*7+1, 366))
    modelo += pulp.lpSum(P[e, d, p] for e in especies for d in dias_semana for p in poligonos) >= parametros['hectareas_dia'] * parametros['plantas_hectarea'] * 5  # 5 días laborales

# 6. Tiempos de tratamiento estacionales
for e in especies:
    for d in dias_planificacion:
        mes = (d - 1) // 30 + 1
        if especies[e]['nopal'] == 1:
            modelo += D[e, d] >= 0.33 * (1.2 if mes in [6,7,8] else 1)  # +20% tiempo en verano
        else:
            modelo += D[e, d] >= 1 * (0.8 if mes in [12,1,2] else 1)  # -20% tiempo en invierno

# ======================
# OPTIMIZACIÓN POR BLOQUES
# ======================
# Dividir el problema en trimestres para manejar la complejidad
for trimestre in range(1, 5):
    dias_trimestre = range((trimestre-1)*90+1, min(trimestre*90+1, 366))
    
    # Resolver por trimestre
    modelo_trim = pulp.LpProblem(f"Optimizacion_Trimestre_{trimestre}", pulp.LpMinimize)
    
    # Copiar variables y restricciones relevantes
    # ... (implementar lógica de división por trimestres)
    
    modelo_trim.solve(pulp.PULP_CBC_CMD(msg=True))
    print(f"Trimestre {trimestre} completado con status: {pulp.LpStatus[modelo_trim.status]}")

# ======================
# RESULTADOS ANUALES
# ======================
if modelo.status == pulp.LpStatusOptimal:
    print("\nSOLUCIÓN ANUAL ÓPTIMA ENCONTRADA")
    print(f"Costo total anual: ${pulp.value(costo_total):,.2f}")
    
    # Exportar resultados mensuales consolidados
    resultados_anuales = []
    for m in meses_planificacion:
        dias_mes = range((m-1)*30+1, min(m*30+1, 366))
        for e in especies:
            total_plantado = sum(pulp.value(P[e,d,p] or 0 for d in dias_mes for p in poligonos))
            if total_plantado > 0:
                resultados_anuales.append({
                    'Mes': m,
                    'Especie': e,
                    'Total_Plantado': total_plantado,
                    'Viajes': pulp.value(N_mensual[m]),
                    'Costo_Mes': sum(
                        pulp.value(X[e,d,v])*10 + pulp.value(P[e,d,p])*20 
                        for d in dias_mes for v in viveros for p in poligonos
                    )
                })
    
    df_anual = pd.DataFrame(resultados_anuales)
    df_anual.to_csv('plan_anual_optimizado.csv', index=False)
    print("Plan anual exportado a 'plan_anual_optimizado.csv'")

/Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/pulp.py:1780: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/samanthabritoozuna/anaconda3/envs/tecs6-arm/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/62d43e4c34024bfaaf1964a0f498e35f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9n/j6tmxz691hx5ggsy_wd2kh040000gn/T/62d43e4c34024bfaaf1964a0f498e35f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 7 RHS
At line 8 BOUNDS
At line 10 ENDATA
Problem MODEL has 0 rows, 1 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 0 rows, 1 columns and 0 elements
Optimal - objective value 0
Optimal objective 0 - 0 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Trimestre 1 completado con status: O

# Heurístico (Hookes)

## Data

In [22]:
# Especies y sus características
especies = {
    'E1': {'nombre': 'Agave lechuguilla', 'volumen': 0.025, 'nopal': 0, 'tratamiento_min': 20},
    'E2': {'nombre': 'Agave salmiana', 'volumen': 0.025, 'nopal': 0, 'tratamiento_min': 20},
    'E3': {'nombre': 'Agave scabra', 'volumen': 0.0125, 'nopal': 0, 'tratamiento_min': 20},
    'E4': {'nombre': 'Agave striata', 'volumen': 0.0171875, 'nopal': 0, 'tratamiento_min': 20},
    'E5': {'nombre': 'Opuntia cantabrigiensis', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E6': {'nombre': 'Opuntia engelmani', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E7': {'nombre': 'Opuntia robusta', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E8': {'nombre': 'Opuntia streptacanta', 'volumen': 0.0171875, 'nopal': 1, 'tratamiento_min': 60},
    'E9': {'nombre': 'Prosopis laevigata', 'volumen': 0.0171875, 'nopal': 0, 'tratamiento_min': 20},
    'E10': {'nombre': 'Yucca filifera', 'volumen': 0.015625, 'nopal': 0, 'tratamiento_min': 20}
}

# Viveros y disponibilidad (en miles de plantas)
viveros = {
    'V1': {'E9': 26.5, 'E10': 26},
    'V2': {'E3': 26, 'E4': 26, 'E5': 17, 'E7': 17},
    'V3': {'E4': 25, 'E5': 18, 'E6': 18, 'E7': 18, 'E8': 18},
    'V4': {'E1': 26, 'E2': 26, 'E3': 26, 'E6': 21, 'E7': 18 }
}

# Polígonos y demanda
poligonos = {
    '27': {'hectareas': 1.28, 'demanda': {'E1': 4, 'E2': 17, 'E3': 4, 'E4': 4, 'E5': 4, 'E6': 3, 'E7': 6, 'E8': 6, 'E9': 8, 'E10': 2}},
    '28': {'hectareas': 6.64, 'demanda': {'E1': 31, 'E2': 148, 'E3': 31, 'E4': 31, 'E5': 37, 'E6': 28, 'E7': 55, 'E8': 48, 'E9': 65, 'E10': 20}},
    '29': {'hectareas': 6.54, 'demanda': {'E1': 30, 'E2': 142, 'E3': 30, 'E4': 30, 'E5': 35, 'E6': 27, 'E7': 53, 'E8': 46, 'E9': 63, 'E10': 19}},
    '30': {'hectareas': 6.76, 'demanda': {'E1': 34, 'E2': 162, 'E3': 34, 'E4': 34, 'E5': 40, 'E6': 31, 'E7': 60, 'E8': 53, 'E9': 71, 'E10': 22}},
    '20': {'hectareas': 1.38, 'demanda': {'E1': 4, 'E2': 21, 'E3': 4, 'E4': 4, 'E5': 5, 'E6': 4, 'E7': 8, 'E8': 7, 'E9': 9, 'E10': 3}},
    '21': {'hectareas': 8.0, 'demanda': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24}},
    '22': {'hectareas': 7.82, 'demanda': {'E1': 36, 'E2': 170, 'E3': 36, 'E4': 36, 'E5': 42, 'E6': 32, 'E7': 63, 'E8': 55, 'E9': 75, 'E10': 23}},
    '23': {'hectareas': 5.53, 'demanda': {'E1': 26, 'E2': 121, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 39, 'E9': 53, 'E10': 16}},
    '24': {'hectareas': 5.64, 'demanda': {'E1': 26, 'E2': 122, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 40, 'E9': 54, 'E10': 16}},
    '18': {'hectareas': 7.11, 'demanda': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21}},
    '17': {'hectareas': 6.11, 'demanda': {'E1': 27, 'E2': 128, 'E3': 27, 'E4': 27, 'E5': 32, 'E6': 24, 'E7': 47, 'E8': 41, 'E9': 56, 'E10': 17}},
    '16': {'hectareas': 5.64, 'demanda': {'E1': 25, 'E2': 118, 'E3': 25, 'E4': 25, 'E5': 29, 'E6': 23, 'E7': 44, 'E8': 38, 'E9': 52, 'E10': 16}},
    '19': {'hectareas': 4.92, 'demanda': {'E1': 21, 'E2': 102, 'E3': 21, 'E4': 21, 'E5': 25, 'E6': 20, 'E7': 38, 'E8': 33, 'E9': 45, 'E10': 14}},
    '25': {'hectareas': 5.05, 'demanda': {'E1': 23, 'E2': 110, 'E3': 23, 'E4': 23, 'E5': 27, 'E6': 21, 'E7': 41, 'E8': 36, 'E9': 48, 'E10': 15}},
    '26': {'hectareas': 4.75, 'demanda': {'E1': 22, 'E2': 106, 'E3': 22, 'E4': 22, 'E5': 26, 'E6': 20, 'E7': 39, 'E8': 34, 'E9': 46, 'E10': 14}},
    '13': {'hectareas': 7.97, 'demanda': {'E1': 36, 'E2': 172, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 63, 'E8': 56, 'E9': 75, 'E10': 23}},
    '31': {'hectareas': 7.34, 'demanda': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 37, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20}},
    '14': {'hectareas': 5.98, 'demanda': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18}},
    '15': {'hectareas': 5.4, 'demanda': {'E1': 25, 'E2': 120, 'E3': 25, 'E4': 25, 'E5': 30, 'E6': 23, 'E7': 44, 'E8': 39, 'E9': 53, 'E10': 16}},
    '7': {'hectareas': 6.28, 'demanda': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18}},
    '8': {'hectareas': 7.6, 'demanda': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21}},
    '9': {'hectareas': 8.0, 'demanda': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24}},
    '10': {'hectareas': 8.0, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '11': {'hectareas': 7.67, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '12': {'hectareas': 1.47, 'demanda': {'E1': 6, 'E2': 31, 'E3': 6, 'E4': 6, 'E5': 8, 'E6': 6, 'E7': 11, 'E8': 10, 'E9': 13, 'E10': 4}},
    '6': {'hectareas': 4.19, 'demanda': {'E1': 18, 'E2': 86, 'E3': 18, 'E4': 18, 'E5': 21, 'E6': 16, 'E7': 32, 'E8': 28, 'E9': 38, 'E10': 12}},
    '2': {'hectareas': 7.52, 'demanda': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 38, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20}},
    '3': {'hectareas': 8.0, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '4': {'hectareas': 8.0, 'demanda': {'E1': 35, 'E2': 168, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 55, 'E9': 74, 'E10': 22}},
    '5': {'hectareas': 7.56, 'demanda': {'E1': 35, 'E2': 167, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 54, 'E9': 74, 'E10': 22}},
    '1': {'hectareas': 5.4, 'demanda': {'E1': 24, 'E2': 115, 'E3': 24, 'E4': 24, 'E5': 29, 'E6': 22, 'E7': 42, 'E8': 37, 'E9': 50, 'E10': 15}},
}




# Parámetros logísticos
parametros = {
    'costo_transporte': 4500,  # $ por viaje
    'costo_almacen':20,
    'costo_plantacion': 20,    # $ por planta
    'capacidad_almacen': 400,  # m³
    'capacidad_tratamiento': 100,  # m³
    'capacidad_camion': 1.238377,  # m³
    'max_plantas_viaje': 8000,
    'hectareas_dia': 5,
    'plantas_hectarea': 2000,
    'jornada_laboral': 6,  # horas
    'costo_gasolina': 23.97,  # $/litro
    'rendimiento_gasolina': 4  # km/litro
}

# Secuencia de plantación (posiciones 1-22)
secuencia_plantacion = [
    'E2', 'E9', 'E8', 'E2', 'E7', 'E3', 'E5', 'E2', 'E6', 'E4',
    'E2', 'E6', 'E9', 'E2', 'E5', 'E1', 'E2', 'E8', 'E10', 'E2',
    'E7', 'E9'
]

# Matriz de supervivencia (días 3-7)
supervivencia = {
    'E1': [0.6, 0.7, 0.8, 0.9, 0.95],
    'E2': [0.65, 0.75, 0.85, 0.92, 0.97],
    'E3': [0.6, 0.7, 0.8, 0.88, 0.95],
    'E4': [0.58, 0.68, 0.78, 0.87, 0.94],
    'E5': [0.5, 0.65, 0.8, 0.9, 0.98],
    'E6': [0.52, 0.66, 0.82, 0.91, 0.98],
    'E7': [0.55, 0.68, 0.83, 0.92, 0.99],
    'E8': [0.54, 0.67, 0.82, 0.91, 0.98],
    'E9': [0.4, 0.5, 0.6, 0.75, 0.85],
    'E10': [0.55, 0.65, 0.75, 0.85, 0.93]
}

In [14]:
import pulp
import pandas as pd
import numpy as np
from itertools import product

# ======================
# DATOS DE ENTRADA
# ======================

# Especies y sus características
especies = {
    'E1': {'nombre': 'Agave lechuguilla', 'volumen': 0.025, 'nopal': 0, 'tratamiento_min': 20},
    'E2': {'nombre': 'Agave salmiana', 'volumen': 0.025, 'nopal': 0, 'tratamiento_min': 20},
    'E3': {'nombre': 'Agave scabra', 'volumen': 0.0125, 'nopal': 0, 'tratamiento_min': 20},
    'E4': {'nombre': 'Agave striata', 'volumen': 0.0171875, 'nopal': 0, 'tratamiento_min': 20},
    'E5': {'nombre': 'Opuntia cantabrigiensis', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E6': {'nombre': 'Opuntia engelmani', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E7': {'nombre': 'Opuntia robusta', 'volumen': 0.015625, 'nopal': 1, 'tratamiento_min': 60},
    'E8': {'nombre': 'Opuntia streptacanta', 'volumen': 0.0171875, 'nopal': 1, 'tratamiento_min': 60},
    'E9': {'nombre': 'Prosopis laevigata', 'volumen': 0.0171875, 'nopal': 0, 'tratamiento_min': 20},
    'E10': {'nombre': 'Yucca filifera', 'volumen': 0.015625, 'nopal': 0, 'tratamiento_min': 20}
}

# Viveros y disponibilidad (en miles de plantas)
viveros = {
    'V1': {'E9': 26.5, 'E10': 26},
    'V2': {'E3': 26, 'E4': 26, 'E5': 17, 'E7': 17},
    'V3': {'E4': 25, 'E5': 18, 'E6': 18, 'E7': 18, 'E8': 18},
    'V4': {'E1': 26, 'E2': 26, 'E3': 26, 'E6': 21, 'E7': 18 }
}

# Polígonos y demanda
poligonos = {
    '27': {'hectareas': 1.28, 'demanda': {'E1': 4, 'E2': 17, 'E3': 4, 'E4': 4, 'E5': 4, 'E6': 3, 'E7': 6, 'E8': 6, 'E9': 8, 'E10': 2}},
    '28': {'hectareas': 6.64, 'demanda': {'E1': 31, 'E2': 148, 'E3': 31, 'E4': 31, 'E5': 37, 'E6': 28, 'E7': 55, 'E8': 48, 'E9': 65, 'E10': 20}},
    '29': {'hectareas': 6.54, 'demanda': {'E1': 30, 'E2': 142, 'E3': 30, 'E4': 30, 'E5': 35, 'E6': 27, 'E7': 53, 'E8': 46, 'E9': 63, 'E10': 19}},
    '30': {'hectareas': 6.76, 'demanda': {'E1': 34, 'E2': 162, 'E3': 34, 'E4': 34, 'E5': 40, 'E6': 31, 'E7': 60, 'E8': 53, 'E9': 71, 'E10': 22}},
    '20': {'hectareas': 1.38, 'demanda': {'E1': 4, 'E2': 21, 'E3': 4, 'E4': 4, 'E5': 5, 'E6': 4, 'E7': 8, 'E8': 7, 'E9': 9, 'E10': 3}},
    '21': {'hectareas': 8.0, 'demanda': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24}},
    '22': {'hectareas': 7.82, 'demanda': {'E1': 36, 'E2': 170, 'E3': 36, 'E4': 36, 'E5': 42, 'E6': 32, 'E7': 63, 'E8': 55, 'E9': 75, 'E10': 23}},
    '23': {'hectareas': 5.53, 'demanda': {'E1': 26, 'E2': 121, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 39, 'E9': 53, 'E10': 16}},
    '24': {'hectareas': 5.64, 'demanda': {'E1': 26, 'E2': 122, 'E3': 26, 'E4': 26, 'E5': 30, 'E6': 23, 'E7': 45, 'E8': 40, 'E9': 54, 'E10': 16}},
    '18': {'hectareas': 7.11, 'demanda': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21}},
    '17': {'hectareas': 6.11, 'demanda': {'E1': 27, 'E2': 128, 'E3': 27, 'E4': 27, 'E5': 32, 'E6': 24, 'E7': 47, 'E8': 41, 'E9': 56, 'E10': 17}},
    '16': {'hectareas': 5.64, 'demanda': {'E1': 25, 'E2': 118, 'E3': 25, 'E4': 25, 'E5': 29, 'E6': 23, 'E7': 44, 'E8': 38, 'E9': 52, 'E10': 16}},
    '19': {'hectareas': 4.92, 'demanda': {'E1': 21, 'E2': 102, 'E3': 21, 'E4': 21, 'E5': 25, 'E6': 20, 'E7': 38, 'E8': 33, 'E9': 45, 'E10': 14}},
    '25': {'hectareas': 5.05, 'demanda': {'E1': 23, 'E2': 110, 'E3': 23, 'E4': 23, 'E5': 27, 'E6': 21, 'E7': 41, 'E8': 36, 'E9': 48, 'E10': 15}},
    '26': {'hectareas': 4.75, 'demanda': {'E1': 22, 'E2': 106, 'E3': 22, 'E4': 22, 'E5': 26, 'E6': 20, 'E7': 39, 'E8': 34, 'E9': 46, 'E10': 14}},
    '13': {'hectareas': 7.97, 'demanda': {'E1': 36, 'E2': 172, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 63, 'E8': 56, 'E9': 75, 'E10': 23}},
    '31': {'hectareas': 7.34, 'demanda': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 37, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20}},
    '14': {'hectareas': 5.98, 'demanda': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18}},
    '15': {'hectareas': 5.4, 'demanda': {'E1': 25, 'E2': 120, 'E3': 25, 'E4': 25, 'E5': 30, 'E6': 23, 'E7': 44, 'E8': 39, 'E9': 53, 'E10': 16}},
    '7': {'hectareas': 6.28, 'demanda': {'E1': 28, 'E2': 132, 'E3': 28, 'E4': 28, 'E5': 33, 'E6': 25, 'E7': 49, 'E8': 43, 'E9': 58, 'E10': 18}},
    '8': {'hectareas': 7.6, 'demanda': {'E1': 33, 'E2': 156, 'E3': 33, 'E4': 33, 'E5': 39, 'E6': 30, 'E7': 58, 'E8': 51, 'E9': 69, 'E10': 21}},
    '9': {'hectareas': 8.0, 'demanda': {'E1': 37, 'E2': 178, 'E3': 37, 'E4': 37, 'E5': 44, 'E6': 34, 'E7': 66, 'E8': 58, 'E9': 78, 'E10': 24}},
    '10': {'hectareas': 8.0, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '11': {'hectareas': 7.67, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '12': {'hectareas': 1.47, 'demanda': {'E1': 6, 'E2': 31, 'E3': 6, 'E4': 6, 'E5': 8, 'E6': 6, 'E7': 11, 'E8': 10, 'E9': 13, 'E10': 4}},
    '6': {'hectareas': 4.19, 'demanda': {'E1': 18, 'E2': 86, 'E3': 18, 'E4': 18, 'E5': 21, 'E6': 16, 'E7': 32, 'E8': 28, 'E9': 38, 'E10': 12}},
    '2': {'hectareas': 7.52, 'demanda': {'E1': 32, 'E2': 151, 'E3': 32, 'E4': 32, 'E5': 38, 'E6': 29, 'E7': 56, 'E8': 49, 'E9': 66, 'E10': 20}},
    '3': {'hectareas': 8.0, 'demanda': {'E1': 36, 'E2': 173, 'E3': 36, 'E4': 36, 'E5': 43, 'E6': 33, 'E7': 64, 'E8': 56, 'E9': 76, 'E10': 23}},
    '4': {'hectareas': 8.0, 'demanda': {'E1': 35, 'E2': 168, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 55, 'E9': 74, 'E10': 22}},
    '5': {'hectareas': 7.56, 'demanda': {'E1': 35, 'E2': 167, 'E3': 35, 'E4': 35, 'E5': 42, 'E6': 32, 'E7': 62, 'E8': 54, 'E9': 74, 'E10': 22}},
    '1': {'hectareas': 5.4, 'demanda': {'E1': 24, 'E2': 115, 'E3': 24, 'E4': 24, 'E5': 29, 'E6': 22, 'E7': 42, 'E8': 37, 'E9': 50, 'E10': 15}},
}




# Parámetros logísticos
parametros = {
    'costo_transporte': 4500,  # $ por viaje
    'costo_plantacion': 20,    # $ por planta
    'capacidad_almacen': 400,  # m³
    'capacidad_tratamiento': 100,  # m³
    'capacidad_camion': 1.238377,  # m³
    'max_plantas_viaje': 8000,
    'hectareas_dia': 5,
    'plantas_hectarea': 2000,
    'jornada_laboral': 6,  # horas
    'costo_gasolina': 23.97,  # $/litro
    'rendimiento_gasolina': 4  # km/litro
}

# Secuencia de plantación (posiciones 1-22)
secuencia_plantacion = [
    'E2', 'E9', 'E8', 'E2', 'E7', 'E3', 'E5', 'E2', 'E6', 'E4',
    'E2', 'E6', 'E9', 'E2', 'E5', 'E1', 'E2', 'E8', 'E10', 'E2',
    'E7', 'E9'
]

# Matriz de supervivencia (días 3-7)
supervivencia = {
    'E1': [0.6, 0.7, 0.8, 0.9, 0.95],
    'E2': [0.65, 0.75, 0.85, 0.92, 0.97],
    'E3': [0.6, 0.7, 0.8, 0.88, 0.95],
    'E4': [0.58, 0.68, 0.78, 0.87, 0.94],
    'E5': [0.5, 0.65, 0.8, 0.9, 0.98],
    'E6': [0.52, 0.66, 0.82, 0.91, 0.98],
    'E7': [0.55, 0.68, 0.83, 0.92, 0.99],
    'E8': [0.54, 0.67, 0.82, 0.91, 0.98],
    'E9': [0.4, 0.5, 0.6, 0.75, 0.85],
    'E10': [0.55, 0.65, 0.75, 0.85, 0.93]
}


## Model

In [25]:
import numpy as np
import pandas as pd
from copy import deepcopy

# Configuración del problema (simplificada para la heurística)
DIAS = 365
ESPECIES = list(especies.keys())
VIVEROS = list(viveros.keys())
POLIGONOS = list(poligonos.keys())

# Estructuras de datos para la solución
class Solucion:
    def __init__(self):
        self.transporte = np.zeros((len(ESPECIES), DIAS, len(VIVEROS)))
        self.plantacion = np.zeros((len(ESPECIES), DIAS, len(POLIGONOS)))
        self.almacen = np.zeros((len(ESPECIES), DIAS+1))
        self.dias_almacen = np.zeros((len(ESPECIES), DIAS))
        self.costo = float('inf')
        self.supervivencia = 0

    def __sub__(self, other):
        """Define subtraction between two Solucion objects."""
        direccion = Solucion()
        direccion.transporte = self.transporte - other.transporte
        direccion.plantacion = self.plantacion - other.plantacion
        direccion.dias_almacen = self.dias_almacen - other.dias_almacen
        return direccion
    
    def evaluar(self):
        self.supervivencia = 0
        self.costo = 0
    
        for e in range(len(ESPECIES)):
            for d in range(DIAS):
                self.almacen[e, d+1] = (
                    self.almacen[e, d] + sum(self.transporte[e, d, :]) 
                    - sum(self.plantacion[e, d, :])
                )
                dias = int(np.clip(self.dias_almacen[e, d], 3, 7))
                prob = supervivencia[ESPECIES[e]][dias - 3]
                self.supervivencia += (
                    sum(self.plantacion[e, d, :]) * prob
                )
                self.costo += (
                    sum(self.transporte[e, d, :]) * parametros['costo_transporte']
                    + self.almacen[e, d] * parametros['costo_almacen']
                    + sum(self.plantacion[e, d, :]) * parametros['costo_plantacion']
                )
        
        self.supervivencia /= np.sum(self.plantacion) + 1e-8


# Algoritmo Hooke-Jeeves adaptado
def hooke_jeeves(max_iter=1000, paso_inicial=1000, reduccion_paso=0.5, tol=1e-4):
    # Inicialización
    paso = paso_inicial
    mejor_sol = Solucion()
    mejor_sol.evaluar()
    
    # Patrón de búsqueda inicial
    patron = [
        {'dim': 'transporte', 'delta': paso},
        {'dim': 'plantacion', 'delta': paso},
        {'dim': 'dias_almacen', 'delta': paso/10}
    ]
    
    iteracion = 0
    while iteracion < max_iter and paso > tol:
        mejor_en_iter = False
        
        # Búsqueda exploratoria
        for dim in patron:
            sol_temp = deepcopy(mejor_sol)
            
            # Exploración positiva
            sol_temp = modificar_solucion(sol_temp, dim, paso)
            sol_temp.evaluar()
            
            if sol_temp.costo < mejor_sol.costo:
                mejor_sol = sol_temp
                mejor_en_iter = True
                continue
                
            # Exploración negativa
            sol_temp = deepcopy(mejor_sol)
            sol_temp = modificar_solucion(sol_temp, dim, -paso)
            sol_temp.evaluar()
            
            if sol_temp.costo < mejor_sol.costo:
                mejor_sol = sol_temp
                mejor_en_iter = True
        
        # Búsqueda pattern
        if mejor_en_iter:
            mejor_sol_anterior = deepcopy(mejor_sol)  # Define mejor_sol_anterior
            while True:
                sol_patron = deepcopy(mejor_sol)
                direccion = sol_patron - mejor_sol_anterior
                
                # Prueba en dirección del patrón
                sol_patron = aplicar_direccion(sol_patron, direccion)
                sol_patron.evaluar()
                
                if sol_patron.costo < mejor_sol.costo:
                    mejor_sol = sol_patron
                else:
                    break
        else:
            paso *= reduccion_paso
        
        iteracion += 1
        print(f"Iter {iteracion}: Costo=${mejor_sol.costo:,.2f}, Supervivencia={mejor_sol.supervivencia:.2%}")
    
    return mejor_sol

def modificar_solucion(sol, dimension, delta):
    """Modifica una solución en una dimensión específica"""
    if dimension['dim'] == 'transporte':
        # Seleccionar aleatoriamente una celda para modificar
        e = np.random.randint(0, len(ESPECIES))
        d = np.random.randint(0, DIAS)
        v = np.random.randint(0, len(VIVEROS))
        
        # Aplicar cambio manteniendo restricciones
        nuevo_valor = sol.transporte[e,d,v] + delta
        sol.transporte[e,d,v] = max(0, nuevo_valor)
        
    elif dimension['dim'] == 'plantacion':
        e = np.random.randint(0, len(ESPECIES))
        d = np.random.randint(0, DIAS)
        p = np.random.randint(0, len(POLIGONOS))
        
        nuevo_valor = sol.plantacion[e,d,p] + delta
        sol.plantacion[e,d,p] = max(0, nuevo_valor)
        
    elif dimension['dim'] == 'dias_almacen':
        e = np.random.randint(0, len(ESPECIES))
        d = np.random.randint(0, DIAS)
        
        nuevo_valor = sol.dias_almacen[e,d] + delta
        sol.dias_almacen[e,d] = np.clip(nuevo_valor, 3, 7)
    
    return sol

def aplicar_direccion(sol, direccion):
    """Aplica una dirección de patrón a la solución"""
    # Implementar lógica para mover la solución en la dirección del patrón
    # manteniendo las restricciones
    return sol

# Función para verificar restricciones
def verificar_restricciones(sol):
    """Verifica que la solución cumpla con todas las restricciones"""
    # 1. Satisfacción de demanda
    for p_idx, poligono in enumerate(POLIGONOS):
        for e_idx, esp in enumerate(ESPECIES):
            total_plantado = sum(sol.plantacion[e_idx,:,p_idx])
            if total_plantado < poligonos[poligono]['demanda'][esp] * 0.9:
                return False
    
    # 2. Capacidad de almacén
    for d in range(DIAS):
        volumen_total = sum(sol.almacen[e,d] * especies[ESPECIES[e]]['volumen'] 
                         for e in range(len(ESPECIES)))
        if volumen_total > parametros['capacidad_almacen']:
            return False
    
    # 3. Restricciones de secuencia (simplificadas)
    return True

# Ejecución del algoritmo
if __name__ == "__main__":
    mejor_solucion = hooke_jeeves(
        max_iter=500,
        paso_inicial=500,
        reduccion_paso=0.8,
        tol=1e-3
    )
    
    # Guardar resultados
    resultados = []
    for e_idx, esp in enumerate(ESPECIES):
        for d in range(DIAS):
            for p_idx, pol in enumerate(POLIGONOS):
                if mejor_solucion.plantacion[e_idx, d, p_idx] > 0:
                    resultados.append({
                        'Dia': d+1,
                        'Especie': esp,
                        'Poligono': pol,
                        'Cantidad': mejor_solucion.plantacion[e_idx, d, p_idx],
                        'Dias_Almacen': mejor_solucion.dias_almacen[e_idx, d],
                        'Prob_Supervivencia': supervivencia[esp][min(int(mejor_solucion.dias_almacen[e_idx, d]),7)-3]
                    })
    
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_heuristico.csv', index=False)
    
    print("\nMejor solución encontrada:")
    print(f"Costo total: ${mejor_solucion.costo:,.2f}")
    print(f"Tasa de supervivencia promedio: {mejor_solucion.supervivencia:.2%}")

Iter 1: Costo=$-630,000.00, Supervivencia=40.00%
Iter 2: Costo=$-3,720,000.00, Supervivencia=47.00%
Iter 3: Costo=$-5,960,000.00, Supervivencia=49.33%
Iter 4: Costo=$-7,990,000.00, Supervivencia=51.50%
Iter 5: Costo=$-9,550,000.00, Supervivencia=52.20%
Iter 6: Costo=$-11,760,000.00, Supervivencia=53.50%
Iter 7: Costo=$-14,220,000.00, Supervivencia=53.29%
Iter 8: Costo=$-16,480,000.00, Supervivencia=53.37%
Iter 9: Costo=$-18,850,000.00, Supervivencia=53.44%
Iter 10: Costo=$-21,970,000.00, Supervivencia=53.50%
Iter 11: Costo=$-25,170,000.00, Supervivencia=53.64%
Iter 12: Costo=$-28,770,000.00, Supervivencia=53.50%
Iter 13: Costo=$-31,360,000.00, Supervivencia=53.54%
Iter 14: Costo=$-34,790,000.00, Supervivencia=53.57%
Iter 15: Costo=$-37,290,000.00, Supervivencia=54.00%
Iter 16: Costo=$-38,590,000.00, Supervivencia=53.12%
Iter 17: Costo=$-40,100,000.00, Supervivencia=53.24%
Iter 18: Costo=$-41,610,000.00, Supervivencia=53.33%
Iter 19: Costo=$-43,410,000.00, Supervivencia=53.42%
Iter 20: 

In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy

# Parámetros
DIAS = 365
ESPECIES = list(especies.keys())
VIVEROS = list(viveros.keys())
POLIGONOS = list(poligonos.keys())
META_HECTAREAS_DIARIA = 5

# Solución
class Solucion:
    def __init__(self):
        self.transporte = np.zeros((len(ESPECIES), DIAS, len(VIVEROS)))
        self.plantacion = np.zeros((len(ESPECIES), DIAS, len(POLIGONOS)))
        self.almacen = np.zeros((len(ESPECIES), DIAS + 1))
        self.dias_almacen = np.zeros((len(ESPECIES), DIAS))
        self.costo = float('inf')
        self.supervivencia = 0.0

    def evaluar(self):
        volumen_alm = np.zeros(DIAS + 1)
        self.almacen = np.zeros((len(ESPECIES), DIAS + 1))
        total_costo = 0
        total_vivas = 0
        total_plantadas = 0

        # Cargar el almacén considerando un día de transporte
        for d in range(DIAS):
            for e_idx, esp in enumerate(ESPECIES):
                for v_idx, v in enumerate(VIVEROS):
                    llegada = d + 1
                    if llegada < DIAS + 1:
                        cantidad = self.transporte[e_idx, d, v_idx]
                        self.almacen[e_idx, llegada] += cantidad

        # Calcular plantaciones, costos y supervivencia
        for d in range(DIAS):
            hectareas_plantadas = 0
            for e_idx, esp in enumerate(ESPECIES):
                for p_idx, pol in enumerate(POLIGONOS):
                    cant = self.plantacion[e_idx, d, p_idx]
                    hectareas = cant / especies[esp]['densidad']
                    hectareas_plantadas += hectareas

                    dias_en_almacen = int(self.dias_almacen[e_idx, d])
                    prob = supervivencia[esp][min(max(dias_en_almacen - 3, 0), 4)]
                    total_vivas += cant * prob
                    total_plantadas += cant

                    # Descontar del almacén
                    if d - dias_en_almacen >= 0:
                        self.almacen[e_idx, d - dias_en_almacen] -= cant

                    # Costos
                    total_costo += cant * (
                        parametros['costo_plantacion'] +
                        parametros['costo_almacen'] * dias_en_almacen
                    )
                    for v_idx, v in enumerate(VIVEROS):
                        total_costo += self.transporte[e_idx, d - dias_en_almacen, v_idx] * parametros['costo_transporte'][v]

            # Penalización por no alcanzar meta diaria
            if abs(hectareas_plantadas - META_HECTAREAS_DIARIA) > 0.1:
                total_costo += 1e6  # penalización fuerte

        self.costo = total_costo
        self.supervivencia = total_vivas / total_plantadas if total_plantadas > 0 else 0

# Modificar solución
def modificar_solucion(sol, dimension, delta):
    if dimension['dim'] == 'transporte':
        e = np.random.randint(0, len(ESPECIES))
        d = np.random.randint(0, DIAS)
        v = np.random.randint(0, len(VIVEROS))
        nuevo_valor = sol.transporte[e, d, v] + delta
        sol.transporte[e, d, v] = max(0, nuevo_valor)

    elif dimension['dim'] == 'plantacion':
        e = np.random.randint(0, len(ESPECIES))
        d = np.random.randint(0, DIAS)
        p = np.random.randint(0, len(POLIGONOS))
        nuevo_valor = sol.plantacion[e, d, p] + delta
        sol.plantacion[e, d, p] = max(0, nuevo_valor)

    elif dimension['dim'] == 'dias_almacen':
        e = np.random.randint(0, len(ESPECIES))
        d = np.random.randint(0, DIAS)
        nuevo_valor = sol.dias_almacen[e, d] + delta
        sol.dias_almacen[e, d] = np.clip(nuevo_valor, 3, 7)

    return sol

# Hooke-Jeeves
def hooke_jeeves(max_iter=1000, paso_inicial=100, reduccion_paso=0.5, tol=1e-3):
    paso = paso_inicial
    mejor_sol = Solucion()
    mejor_sol.evaluar()

    patron = [
        {'dim': 'transporte', 'delta': paso},
        {'dim': 'plantacion', 'delta': paso},
        {'dim': 'dias_almacen', 'delta': paso / 10}
    ]

    iteracion = 0
    while iteracion < max_iter and paso > tol:
        mejor_en_iter = False

        for dim in patron:
            sol_temp = deepcopy(mejor_sol)
            sol_temp = modificar_solucion(sol_temp, dim, paso)
            sol_temp.evaluar()
            if sol_temp.costo < mejor_sol.costo:
                mejor_sol = sol_temp
                mejor_en_iter = True
                continue

            sol_temp = deepcopy(mejor_sol)
            sol_temp = modificar_solucion(sol_temp, dim, -paso)
            sol_temp.evaluar()
            if sol_temp.costo < mejor_sol.costo:
                mejor_sol = sol_temp
                mejor_en_iter = True

        if not mejor_en_iter:
            paso *= reduccion_paso

        iteracion += 1
        print(f"Iter {iteracion}: Costo=${mejor_sol.costo:,.2f}, Supervivencia={mejor_sol.supervivencia:.2%}")

    return mejor_sol

# Exportar resultados como plan de trabajo
def exportar_plan(sol):
    plan = []
    for d in range(DIAS):
        for e_idx, esp in enumerate(ESPECIES):
            for p_idx, pol in enumerate(POLIGONOS):
                cant = sol.plantacion[e_idx, d, p_idx]
                if cant > 0:
                    dias = int(sol.dias_almacen[e_idx, d])
                    plan.append({
                        'Dia': d + 1,
                        'Especie': esp,
                        'Poligono': pol,
                        'Cantidad': cant,
                        'Dias_Almacen': dias,
                        'Prob_Supervivencia': supervivencia[esp][min(max(dias - 3, 0), 4)]
                    })

    df_plan = pd.DataFrame(plan)
    df_plan.to_csv('plan_reforestacion.csv', index=False)
    print("Plan exportado a 'plan_reforestacion.csv'")

# Main
if __name__ == "__main__":
    mejor_solucion = hooke_jeeves(
        max_iter=300,
        paso_inicial=100,
        reduccion_paso=0.7,
        tol=1e-3
    )

    exportar_plan(mejor_solucion)
    print(f"\nCosto total: ${mejor_solucion.costo:,.2f}")
    print(f"Supervivencia promedio: {mejor_solucion.supervivencia:.2%}")


KeyError: 'densidad'

In [28]:
import random
from collections import defaultdict

In [29]:
# Distancias simuladas (en km)
distancias = {}
viveros_keys = viveros.keys()
for v in viveros_keys:
    distancias[v] = {}
    for p in poligonos.keys():
        distancias[v][p] = random.randint(5, 50)  # entre 5 y 50 km

almacen = '18'

# Asignación heurística: cubrir demanda si hay oferta
asignacion = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))  # [vivero][polígono][especie] = cantidad

# Clonar disponibilidad
disponibilidad = {v: viveros[v].copy() for v in viveros}

for poligono, datos in poligonos.items():
    demanda = datos['demanda']
    for especie, cantidad in demanda.items():
        restante = cantidad
        # Buscar viveros que tienen esta especie
        viveros_con_especie = [v for v in viveros if especie in viveros[v] and disponibilidad[v][especie] > 0]
        # Ordenar por cercanía al almacén o al polígono
        viveros_ordenados = sorted(viveros_con_especie, key=lambda v: distancias[v][almacen])
        for v in viveros_ordenados:
            disponible = disponibilidad[v][especie]
            asignado = min(disponible, restante)
            if asignado > 0:
                asignacion[v][poligono][especie] += asignado
                disponibilidad[v][especie] -= asignado
                restante -= asignado
            if restante == 0:
                break

# Cálculo de costos logísticos: se asume transporte desde vivero → almacén (polígono 18) → polígono destino
costos_transporte = {}
costo_km = 2  # valor ficticio por km*unidad

for v in asignacion:
    for p in asignacion[v]:
        for e in asignacion[v][p]:
            cantidad = asignacion[v][p][e]
            distancia_total = distancias[v][almacen] + distancias[v][p]
            costo_total = cantidad * distancia_total * costo_km
            costos_transporte[(v, p, e)] = costo_total

# Mostrar resultados
print("Asignación de especies (vivero → polígono):\n")
for v in asignacion:
    for p in asignacion[v]:
        for e in asignacion[v][p]:
            print(f"Vivero {v} envía {asignacion[v][p][e]} de {e} al polígono {p}")

print("\nCostos de transporte estimados:\n")
costo_total_global = 0
for clave, costo in costos_transporte.items():
    v, p, e = clave
    print(f"{v} → {p} ({e}): ${round(costo, 2)}")
    costo_total_global += costo

print(f"\nCosto total estimado del transporte: ${round(costo_total_global, 2)}")

Asignación de especies (vivero → polígono):

Vivero V4 envía 4 de E1 al polígono 27
Vivero V4 envía 17 de E2 al polígono 27
Vivero V4 envía 4 de E3 al polígono 27
Vivero V4 envía 3 de E6 al polígono 27
Vivero V4 envía 6 de E7 al polígono 27
Vivero V4 envía 22 de E1 al polígono 28
Vivero V4 envía 9 de E2 al polígono 28
Vivero V4 envía 22 de E3 al polígono 28
Vivero V4 envía 18 de E6 al polígono 28
Vivero V4 envía 12 de E7 al polígono 28
Vivero V2 envía 4 de E4 al polígono 27
Vivero V2 envía 4 de E5 al polígono 27
Vivero V2 envía 9 de E3 al polígono 28
Vivero V2 envía 22 de E4 al polígono 28
Vivero V2 envía 13 de E5 al polígono 28
Vivero V2 envía 17 de E7 al polígono 28
Vivero V2 envía 17 de E3 al polígono 29
Vivero V3 envía 6 de E8 al polígono 27
Vivero V3 envía 9 de E4 al polígono 28
Vivero V3 envía 18 de E5 al polígono 28
Vivero V3 envía 10 de E6 al polígono 28
Vivero V3 envía 18 de E7 al polígono 28
Vivero V3 envía 12 de E8 al polígono 28
Vivero V3 envía 16 de E4 al polígono 29
Viver

In [33]:
# Flujos por etapa
flujo_produccion = defaultdict(lambda: defaultdict(float))  # vivero -> especie -> cantidad enviada
flujo_almacen = defaultdict(float)     # especie -> cantidad recibida en almacén
flujo_distribucion = defaultdict(lambda: defaultdict(float))  # polígono -> especie -> cantidad recibida

# Costos por etapa
costos_produccion = 0
costos_distribucion = 0

# Copia de disponibilidad
stock = {v: viveros[v].copy() for v in viveros}

# Heurística completa
for pol, datos in poligonos.items():
    if pol == almacen:
        continue
    for esp, cant in datos['demanda'].items():
        restante = cant
        # Buscar viveros disponibles con esa especie
        disponibles = [v for v in viveros if esp in viveros[v] and stock[v][esp] > 0]
        # Priorizar por distancia al almacén
        disponibles_orden = sorted(disponibles, key=lambda v: distancias[v][almacen])
        for v in disponibles_orden:
            disponible = stock[v][esp]
            asignado = min(disponible, restante)
            if asignado == 0:
                continue

            # Actualizar flujos
            stock[v][esp] -= asignado
            flujo_produccion[v][esp] += asignado
            flujo_almacen[esp] += asignado
            flujo_distribucion[pol][esp] += asignado

            # Costos
            d1 = distancias[v][almacen]
            d2 = distancias[v][pol]
            costos_produccion += asignado * d1 * costo_km
            costos_distribucion += asignado * d2 * costo_km

            restante -= asignado
            if restante <= 0:
                break

# Mostrar resumen del supply chain
print("📦 FLUJO DE PRODUCCIÓN:")
for v in flujo_produccion:
    for e in flujo_produccion[v]:
        print(f"{v} → almacén: {flujo_produccion[v][e]} unidades de {e}")

print("\n🏬 FLUJO DE ALMACÉN (Total recibido):")
for e in flujo_almacen:
    print(f"Almacén recibe {flujo_almacen[e]} unidades de {e}")

print("\n🚚 DISTRIBUCIÓN FINAL A POLÍGONOS:")
for p in flujo_distribucion:
    for e in flujo_distribucion[p]:
        print(f"Almacén → Polígono {p}: {flujo_distribucion[p][e]} unidades de {e}")

print("\n COSTOS:")
print(f"Producción (vivero → almacén): ${round(costos_produccion,2)}")
print(f"Distribución (almacén → polígonos): ${round(costos_distribucion,2)}")
print(f" Costo total de la cadena de suministro: ${round(costos_produccion + costos_distribucion,2)}")

📦 FLUJO DE PRODUCCIÓN:
V4 → almacén: 26.0 unidades de E1
V4 → almacén: 26.0 unidades de E2
V4 → almacén: 26.0 unidades de E3
V4 → almacén: 21.0 unidades de E6
V4 → almacén: 18.0 unidades de E7
V2 → almacén: 26.0 unidades de E4
V2 → almacén: 17.0 unidades de E5
V2 → almacén: 26.0 unidades de E3
V2 → almacén: 17.0 unidades de E7
V3 → almacén: 18.0 unidades de E8
V3 → almacén: 25.0 unidades de E4
V3 → almacén: 18.0 unidades de E5
V3 → almacén: 18.0 unidades de E6
V3 → almacén: 18.0 unidades de E7
V1 → almacén: 26.5 unidades de E9
V1 → almacén: 26.0 unidades de E10

🏬 FLUJO DE ALMACÉN (Total recibido):
Almacén recibe 26.0 unidades de E1
Almacén recibe 26.0 unidades de E2
Almacén recibe 52.0 unidades de E3
Almacén recibe 51.0 unidades de E4
Almacén recibe 35.0 unidades de E5
Almacén recibe 39.0 unidades de E6
Almacén recibe 53.0 unidades de E7
Almacén recibe 18.0 unidades de E8
Almacén recibe 26.5 unidades de E9
Almacén recibe 26.0 unidades de E10

🚚 DISTRIBUCIÓN FINAL A POLÍGONOS:
Almacén 